# Todo List

- [x] completer le fichier function_preprocess
- [x] importer le preprocess qui reste dans #3
- [x] dans function_preprocess, reprendre le traitement de failure_type
- [ ]
- [ ] 

_____________________________________________________________________________________________________________

In [1]:
FILE_PATH = '../data/predictive_maintenance_A.csv'

import pandas as pd

import sys
sys.path.insert(0, '../Src/')

from Function_preprocess import preprocess

df = pd.read_csv(FILE_PATH, index_col='UDI')

df_train, dict_target = preprocess(df)


In [ ]:
df_train.head()

_______________________________________________________________________________________________________________________

### Import

In [2]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


In [3]:
import warnings

warnings.filterwarnings(action='ignore')

In [4]:
FILE_PATH = '../data/predictive_maintenance_A.csv'

### Read file data

In [5]:
df = pd.read_csv(FILE_PATH, index_col='UDI')
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type,Difference temperature [K],Power
UDI,,,,,,,,,,
1,M,298.1,308.6,1551,42.8,0,0,No Failure,10.5,66382.8
2,L,298.2,308.7,1408,46.3,3,0,No Failure,10.5,65190.4
3,L,298.1,308.5,1498,49.4,5,0,No Failure,10.4,74001.2
4,L,298.2,308.6,1433,39.5,7,0,No Failure,10.4,56603.5
5,L,298.2,308.7,1408,40.0,9,0,No Failure,10.5,56320.0


Delete rows with 'Random Failures' in type column with target == 0

In [6]:
df_tmp = df[df['Target']==0]
df_tmp['Failure Type'].value_counts()

Failure Type
No Failure         9643
Random Failures      18
Name: count, dtype: int64

In [7]:
idx = df_tmp[df_tmp['Failure Type'] == 'Random Failures'].index
df.drop(index=idx, axis=0, inplace=True)

Delete rows with 'No Failure' in type column with target == 1

In [8]:
df_tmp = df[df['Target']==1]
df_tmp['Failure Type'].value_counts()

Failure Type
Heat Dissipation Failure    112
Power Failure                95
Overstrain Failure           78
Tool Wear Failure            45
No Failure                    9
Name: count, dtype: int64

In [9]:
idx = df_tmp[df_tmp['Failure Type']=='No Failure'].index
df.drop(index=idx, axis=0, inplace=True)

In [10]:
df.shape

(9973, 10)

In [27]:
!pip install --upgrade tensorflow --user

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/7c/3c/049400232ee2897d613db9009e42269417166c3f8519d46e5c97b6f9e206/tensorflow-2.15.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl.metadata (3.6 kB)
  Obtaining dependency information for tensorboard<2.16,>=2.15 from https://files.pythonhosted.org/packages/6e/0c/1059a6682cf2cc1fcc0d5327837b5672fe4f5574255fa5430d0a8ceb75e9/tensorboard-2.15.1-py3-none-any.whl.metadata
  Using cached tensorboard-2.15.1-py3-none-any.whl.metadata (1.7 kB)
  Obtaining dependency information for google-auth-oauthlib<2,>=0.5 from https://files.pythonhosted.org/packages/71/bf/9e125754d1adb3bc4bd206c4e5df756513b1d23675ac06caa471278d1f3f/google_auth_oauthlib-1.2.0-py2.py3-none-any.whl.metadata
  Using cached google_auth_oauthlib-1.2.0-py2.py3-none-any.whl.metadata (2.7 kB)
Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl (2.1 kB)
Using cached tensorboard-2.15.1-py3

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [38]:

#from tensorflow import keras
#from tensorflow.python.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.python.keras.utils import to_categorical
a =to_categorical(df['Failure Type'], num_classes=df['Failure Type'].nunique())
print(a)

ImportError: cannot import name 'to_categorical' from 'tensorflow.python.keras.utils' (c:\Users\yanni\anaconda3\envs\python_3_10\lib\site-packages\tensorflow\python\keras\utils\__init__.py)

# Data Formatting
## Conversion 'Failure Type' variable to discret value

In [ ]:
lst_failure = df['Failure Type'].unique()
dict_failure = {}
for i, failure in enumerate(lst_failure):
    dict_failure[failure] = i
df['Failure Type'].map(dict_failure)  

## List of categorical variables
Memorize features who are 'object' type, exclude target variables 

In [ ]:
cat_cols = df.select_dtypes(include='O').columns.to_list()
cat_cols.remove('Failure Type')
#cat_cols.append('Target')
cat_cols

## List of numerical variables
Memorize features who are 'numerical' type, exclude target variables 

In [ ]:
num_cols = df.select_dtypes(exclude='O').columns.to_list()
num_cols.remove('Target')
num_cols

## Preprocessing of variables

#### Pipeline creation

Define different preprocessing to dataset
1. OnehotEncoder process from list of variables in 'cat_cols'
2. StandardScaler process from list of variables in 'num_cols'
3. OrdinalEncoder process for 'Failure Type' variables

In [ ]:
ct = ColumnTransformer([
    ("onehot", OneHotEncoder(), cat_cols),
    ("scale", StandardScaler(), num_cols),
    ("fail_type", OrdinalEncoder(), ['Target'])
])


Apply preprocess transformer and create dataframe 'df_preprocess' 

In [ ]:
df.drop(columns=['Target'], inplace=True)
df.rename(columns={'Failure Type': 'Target'}, inplace=True)
df_preprocess = pd.DataFrame(ct.fit_transform(df), columns=ct.get_feature_names_out())


In [ ]:

df_preprocess.dropna(axis=0, inplace=True)

Create dictionnary with index and value of 'target' feature

In [ ]:
cat_=ct.named_transformers_['fail_type'].categories_
dict_target = {}
dict_target_inv = {}
for i, c in enumerate(cat_[0]):
    dict_target[c] = i
    dict_target_inv[i] = c


Split dataframe into dataset for training and dataset for validation

In [ ]:
X_, X_val, y_, y_val = train_test_split(
    df_preprocess.drop(columns=['fail_type__Target']), 
    df_preprocess['fail_type__Target'], 
    test_size=0.2,
    stratify=df_preprocess['fail_type__Target'],
    random_state=42)

In [ ]:
df_train = pd.concat([X_, y_], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)

Save dataframe

In [ ]:
df_train.to_csv('../data/predictive_maintenance_preprocess.csv')
df_val.to_csv('../data/predictive_maintenance_validation.csv')

Save dictionary of target

In [ ]:
with open('../data/dict_target.json', 'w') as f:
    json.dump(dict_target_inv, f)